# Download SQuAD Dataset and preprocess
- Download Train + eval
- tokenize data and write to separate files (context, question, answers)

## Download

In [1]:
import importlib
import preprocess
importlib.reload(preprocess)
from preprocess import download_squad_dataset, process_split, write_to_files

train, eval = download_squad_dataset()

In [2]:
print(len(train["data"]))
print(len(eval["data"]))

442
48


## Tokenization

- tokenization function (Stanford coreNLP tokenizer python only alternative)
- mapping function: (context, context_tokens) -> dictionary mapping char indices to tokens: <br>
example ("this is a test", [this, is, a, test]) ---> 0,1,2,3 -> ("this",0), 5,6 -> ("is",1), ... etc. 

In [3]:
import stanza

nlp = stanza.Pipeline(lang = "en", processors="tokenize", tokenize_pretokenized = False)
eval_dataset = process_split(eval, nlp)
e_context_tokens, e_question_tokens, e_answer_tokens, e_span_tokens = write_to_files(eval_dataset, "eval")

/home/leon1/dnlp2025/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-14 11:30:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-06-14 11:30:03 INFO: Downloaded file to /home/leon1/stanza_resources/resources.json
2025-06-14 11:30:03 WARNING: Language en package default expects mwt, which has been added
2025-06-14 11:30:04 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2025-06-14 11:30:04 INFO: Using device: cpu
2025-06-14 11:30:04 INFO: Loading: tokenize
2025-06-14 11:30:05 INFO: Loading: mwt
2025-06-14 11:30:05 INFO: Done loading pro

mappingissues: 0
spanissues: 0
tokenissues: 0


In [4]:
#train_dataset = process_split(train)
e_context_tokens, e_question_tokens, e_answer_tokens, e_span_tokens = write_to_files(eval_dataset, "eval")
#t_context_tokens, t_question_tokens, t_answer_tokens, t_span_tokens = write_to_files(train_dataset, "train")

## Map tokens to embedding indices

- load GloVe embeddings
- map vocabulary to embedding indices


In [5]:
import numpy as np
import os 

glove_path = os.path.abspath(os.path.dirname(os.getcwd())) + "/glove_embeddings/glove.840B.300d.txt"
print(glove_path) 
assert os.path.exists(glove_path), ("glove embeddings file missing! Please download the correct embeddings and place them into the glove_embeddings directory")
embedding_index = {}
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        vals = line.split(' ')
        word = vals[0]
        coefs = np.asarray(vals[1:], dtype='float32')
        embedding_index[word] = coefs

print("Done! ", len(embedding_index),"words loaded")

/home/leon1/dnlp2025/glove_embeddings/glove.840B.300d.txt
Done!  400000 words loaded


In [6]:
word2idx = {}
idx2word = []
embedding_dim = 300
embedding_matrix = []

word2idx["[PAD]"] = 0
word2idx["[UNK]"] = 1
idx2word.append("[PAD]")
idx2word.append("[UNK]")
embedding_matrix.append(np.zeros(embedding_dim, dtype='float32'))
embedding_matrix.append(np.zeros(embedding_dim, dtype='float32'))

def get_or_create_index(token):
    token_lower = token.lower()
    if(token_lower) in word2idx:
        return word2idx[token_lower]
    else:
        idx = len(word2idx)
        word2idx[token_lower] = idx
        idx2word.append(token_lower)
        if token_lower in embedding_index:
            embedding_matrix.append(embedding_index[token_lower])
        else:
            embedding_matrix.append(np.random.normal(scale=0.01, size=embedding_dim))
        return idx

In [7]:
sen_idxs = []
#do this for every token in contexts,question and answers
all_tokens = []
all_tokens.extend(e_context_tokens)
all_tokens.extend(e_question_tokens)
all_tokens.extend(e_answer_tokens)
print(len(all_tokens))
i = 0
for tokens in all_tokens:
    if tokens is None:
        continue
    idx = [get_or_create_index(t) for t in tokens.split()]
    sen_idxs.append(idx)
print(len(sen_idxs))
print(len(word2idx))
print(len(idx2word))
print(sen_idxs[0])

31338
31338
25764
25764
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 12, 15, 8, 16, 17, 18, 19, 20, 12, 21, 22, 23, 12, 7, 8, 24, 17, 25, 19, 13, 26, 27, 28, 12, 15, 8, 24, 17, 29, 19, 13, 30, 31, 32, 33, 34, 10, 35, 36, 37, 2, 3, 38, 23, 12, 9, 5, 39, 40, 41, 42, 43, 44, 43, 45, 46, 47, 48, 12, 49, 50, 51, 52, 45, 53, 54, 43, 55, 23, 56, 57, 5, 12, 58, 2, 3, 43, 12, 16, 59, 12, 60, 61, 62, 60, 63, 64, 65, 66, 67, 68, 43, 56, 69, 56, 70, 71, 12, 72, 14, 73, 74, 2, 3, 9, 63, 75, 76, 17, 77, 78, 12, 9, 79, 80, 81, 82, 56, 60, 2, 3, 83, 60, 19, 43, 84, 85, 12, 86, 87, 88, 89, 12, 90, 76, 4, 23]


In [8]:
embedding_matrix = np.array(embedding_matrix, dtype='float32')
embedding_matrix.shape

(25764, 300)

In [9]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, hidden_dim, embedding_matrix, dropout_ratio):
        super().__init__()
        self.hidden_dim = hidden_dim
        vocab_size, embedding_dim = embedding_matrix.shape
        embedding_tensor = torch.tensor(embedding_matrix, dtype=torch.float)
        
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 1, batch_first=True, dropout=dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

        self.w = nn.Linear(hidden_dim, hidden_dim)
        self.b = nn.Parameter(torch.zeros(hidden_dim))

        self.sentinel = nn.Parameter(torch.randn(1,hidden_dim))

    def encode_sequence(self, idxs, mask):
        lengths = mask.sum(dim=1)  # [batch]
        sorted_lens, sorted_idx = lengths.sort(descending=True)
        _, orig_idx = sorted_idx.sort()

        # Sort sequences for packing
        idxs_sorted = idxs[sorted_idx]
        emb = self.embedding(idxs_sorted)
        packed = pack_padded_sequence(emb, sorted_lens.cpu(), batch_first=True, enforce_sorted=True)

        # LSTM encoding
        packed_out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)  # [batch, max_len, hidden]
        out = self.dropout(out)
        out = out[orig_idx]  # restore original order

        # Insert sentinel at end-of-sequence index for each example
        batch_size = out.size(0)
        sentinel_expanded = self.sentinel.expand(batch_size, 1, self.hidden_dim)  # [batch, 1, hidden]

        out_with_sentinel = torch.cat([out, torch.zeros_like(sentinel_expanded)], dim=1)  # [batch, max_len+1, hidden]
        lens = lengths.long().unsqueeze(1).unsqueeze(2).expand(-1, 1, self.hidden_dim)  # [batch, 1, hidden]
        out_with_sentinel = out_with_sentinel.scatter(1, lens, sentinel_expanded)

        return out_with_sentinel  # [batch, seq_len + 1, hidden]

    def forward(self, doc_idxs, doc_mask, q_idxs, q_mask):
        """
        doc_idxs/q_idxs: [batch, seq_len]
        doc_mask/q_mask: [batch, seq_len]
        """
        D = self.encode_sequence(doc_idxs, doc_mask)  # [batch, m+1, hidden]
        Q_prime = self.encode_sequence(q_idxs, q_mask)  # [batch, n+1, hidden]

        # Nonlinear projection: Q = tanh(W * Q′ + b)
        Q = torch.tanh(self.w(Q_prime) + self.b)  # [batch, n+1, hidden]

        return D, Q       

## Small Encoder test

In [10]:
import torch
import numpy as np

# Dummy vocab
word2idx_test = {
    "[PAD]": 0, "[UNK]": 1,
    "the": 2, "quick": 3, "brown": 4, "fox": 5, "jumps": 6, "over": 7, "lazy": 8, "dog": 9
}

# Random embedding matrix for vocab (vocab_size x emb_dim)
vocab_size = len(word2idx_test)
embedding_dim = 50
embedding_matrix_test = np.random.uniform(-0.1, 0.1, (vocab_size, embedding_dim))

# Dummy inputs
# Document: "the quick brown fox jumps"
# Question: "the fox"
doc_tokens = [2, 3, 4, 5, 6]
q_tokens = [2, 5]

# Padding to max length
doc_max_len = 6
q_max_len = 4
doc_input = [doc_tokens + [0] * (doc_max_len - len(doc_tokens))]  # batch size 1
q_input = [q_tokens + [0] * (q_max_len - len(q_tokens))]

# Masks (1 for real tokens, 0 for padding)
doc_mask = [[1]*len(doc_tokens) + [0]*(doc_max_len - len(doc_tokens))]
q_mask = [[1]*len(q_tokens) + [0]*(q_max_len - len(q_tokens))]

# Convert to tensors
doc_idxs = torch.tensor(doc_input)      # [1, 6]
doc_mask = torch.tensor(doc_mask)       # [1, 6]
q_idxs = torch.tensor(q_input)          # [1, 4]
q_mask = torch.tensor(q_mask)           # [1, 4]

hidden_size = 64
encoder = Encoder(hidden_size, embedding_matrix_test, 0)

# Run encoder
D, Q = encoder(doc_idxs, doc_mask, q_idxs, q_mask)

# Outputs
print("Document encoding shape:", D.shape)  # [1, m+1, 64]
print("Question encoding shape:", Q.shape)      # [1, n+1, 64]

print("\nSentinel vector (document):", D[0, -1])
print("Sentinel vector (question):", Q[0, -1])


Document encoding shape: torch.Size([1, 6, 64])
Question encoding shape: torch.Size([1, 3, 64])

Sentinel vector (document): tensor([-0.5178, -1.9964, -0.0055, -0.1947, -0.0296,  2.8728, -0.9718,  0.7252,
         2.3006,  0.6986, -0.7614, -0.5578,  0.3154, -0.3911, -0.2983,  1.4246,
        -0.8788, -1.3894, -0.9157, -0.4858,  0.7712,  1.7142,  0.4245,  0.9347,
        -0.0751,  1.4394,  0.1151,  0.8613,  0.0950,  1.2354, -0.3196, -0.8543,
        -0.8129, -0.0994,  0.9303,  1.2679,  1.1180, -1.1823, -0.5686, -0.1963,
         2.7319,  1.0414,  0.4065,  0.1175, -0.1640, -1.2357,  2.2861, -0.3782,
        -0.7519, -1.9009,  0.2609, -1.7670, -1.3268, -0.6697,  2.0614, -1.7673,
         0.0242, -0.9860,  1.3433, -0.2099, -0.4606,  1.2304,  0.2368, -0.2514],
       grad_fn=<SelectBackward0>)
Sentinel vector (question): tensor([ 0.7315, -0.2916, -0.7098,  0.7575, -0.3021,  0.4009,  0.6846, -0.2770,
        -0.6569, -0.6911, -0.7479,  0.0341,  0.1462,  0.6462, -0.0092, -0.4113,
         0.2

In [ ]:
import torch.nn.functional as functional

class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        return out

class CoattentionEncoder(nn.Module):
    def __init__(self, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size

        self.brnn = BRNN(input_size=3 * hidden_size,
                         hidden_size=hidden_size,
                         num_layers=num_layers)

    #2.2 Coattention Encoder
    def forward(self, D, Q):
        #Affinity matrix
        L = torch.bmm(Q, torch.transpose(D, 1, 2)) 

        #Attention weights
        AQ = functional.softmax(L, dim=1)         
        AD = functional.softmax(torch.transpose(L, 1, 2), dim=1)  

        #Context Summaries
        CQ = torch.bmm(AQ, D) 
        Q_combined = torch.cat([Q, CQ], dim=2)   
        CD = torch.bmm(AD, Q_combined)

        #BRNN
        return self.brnn(torch.cat([D, CD], dim=2)) 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
coattention_enc = CoattentionEncoder(hidden_size).to(device)

U = coattention_enc(D, Q)  

print(U)

tensor([[[-3.8170e-02,  3.0651e-03, -1.6778e-02, -3.4676e-02, -3.1467e-02,
           2.7263e-03, -1.0733e-02,  3.6497e-02,  6.5026e-02,  2.1943e-02,
           4.1403e-02, -6.9917e-03,  6.4237e-02,  1.0203e-01, -1.5333e-02,
           2.2456e-02, -3.4000e-02,  1.9732e-03,  3.8344e-02, -8.9095e-02,
           2.2353e-03, -2.6979e-03,  1.5462e-02,  3.8507e-02, -6.0910e-02,
           2.6456e-02, -3.0254e-02, -5.7672e-02,  5.9515e-02,  8.8467e-03,
          -7.0999e-02,  7.2277e-02, -1.2743e-02,  5.5897e-02,  3.0679e-02,
          -2.8931e-02, -1.3734e-02, -4.9860e-03, -2.1400e-03,  1.7617e-02,
           1.5752e-02,  3.0865e-02, -2.8231e-02, -3.9334e-02, -8.9450e-02,
          -2.5320e-02, -8.4639e-02,  2.7392e-02,  6.7270e-02,  4.6448e-02,
           1.8610e-03,  2.2511e-03, -2.6907e-02, -4.2986e-02, -1.0375e-02,
           1.3769e-03,  2.1865e-02, -5.9250e-02,  3.6065e-02, -6.6762e-02,
           2.3081e-02, -6.9961e-02,  1.9613e-02,  9.0126e-03,  2.9161e-02,
           4.5725e-02, -2